In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import pandas as pd
# import numpy as np

In [3]:
# train_trx = pd.read_parquet("./Data/trx_train.parquet")
# test_trx = pd.read_parquet("./Data/trx_test.parquet")

In [4]:
# df = pd.concat([train_trx, test_trx])

In [5]:
# df.to_parquet(f'./Data/trx.parquet', engine='pyarrow')

In [6]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T


data_path = 'data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

24/06/13 20:58:58 WARN Utils: Your hostname, dimk-B450M-DS3H resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface eno1)
24/06/13 20:58:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 20:58:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/13 20:58:59 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


[('spark.driver.memoryOverhead', '4g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.port', '39605'),
 ('spark.app.id', 'local-17183

In [7]:
trx_train = spark.read.options(header=True, inferSchema=True).parquet("./Data/trx_train.parquet")
trx_train.show(2)

+--------------------+---------+--------------------+----------+-------------+--------+----------+----------+----------+----------+----------+----------+----------+----------+
|          event_time|   amount|           client_id|event_type|event_subtype|currency|src_type11|src_type12|dst_type11|dst_type12|src_type21|src_type22|src_type31|src_type32|
+--------------------+---------+--------------------+----------+-------------+--------+----------+----------+----------+----------+----------+----------+----------+----------+
|2022-08-07 09:39:...|700520.94|b1e0c71f4825c4c6b...|        54|           55|    11.0|      19.0|     344.0|     364.0|   22652.0|   29127.0|      70.0|     568.0|      77.0|
|2022-09-10 14:06:...|521687.12|b1e0c71f4825c4c6b...|        54|           55|    11.0|      19.0|     344.0|     364.0|   22652.0|   29127.0|      70.0|     568.0|      77.0|
+--------------------+---------+--------------------+----------+-------------+--------+----------+----------+----------+

In [8]:
trx_test = spark.read.options(header=True, inferSchema=True).parquet("./Data/trx_test.parquet")
df = spark.read.options(header=True, inferSchema=True).parquet("./Data/trx.parquet")
df.show(2)

+--------------------+---------+--------------------+----------+-------------+--------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+
|          event_time|   amount|           client_id|event_type|event_subtype|currency|src_type11|src_type12|dst_type11|dst_type12|src_type21|src_type22|src_type31|src_type32|__index_level_0__|
+--------------------+---------+--------------------+----------+-------------+--------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+
|2022-04-29 13:56:...|156129.73|51f88b707d9c8766c...|        56|            9|    11.0|     109.0|      74.0|     364.0|   22652.0|   46339.0|      43.0|    2423.0|      25.0|                0|
|2022-07-19 12:28:...|357370.22|51f88b707d9c8766c...|        56|            9|    11.0|     109.0|      74.0|     364.0|   22652.0|   46339.0|      43.0|    2423.0|      25.0|                1|
+--------------------+--------

In [9]:
from ptls.preprocessing import PysparkDataPreprocessor

preprocessor = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="event_time",
    event_time_transformation="dt_to_timestamp",
    cols_category=["event_type",
                   "event_subtype",
                   "currency",
                   "src_type11",
                   "src_type12",
                   "dst_type11",
                   "dst_type12",
                   "src_type21",
                   "src_type22",
                   "src_type31",
                   "src_type32"],
    cols_identity="amount",

)


In [10]:
train_dataset = preprocessor.fit_transform(trx_train).persist()
train_dataset.count()

24/06/13 20:49:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:49:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:49:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:49:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:49:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:49:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 2

855627

In [11]:
train_dataset.show(2)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           client_id|          event_time|              amount|          event_type|       event_subtype|            currency|          src_type11|          src_type12|          dst_type11|          dst_type12|          src_type21|          src_type22|          src_type31|          src_type32|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0002f001b84dcf00b...|[1641016974, 1641...|[26223.332, 37334...|[8, 21, 21, 2, 2,...|[8, 20, 20, 2, 2,...|[1, 1,

In [13]:
test_dataset = preprocessor.fit_transform(trx_test).persist()
test_dataset.count()

24/06/13 20:53:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:53:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:53:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:53:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:53:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:53:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 2

225744

In [10]:
df = preprocessor.fit_transform(df).persist()
df.count()

24/06/13 20:59:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:59:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:59:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:59:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:59:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 20:59:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/13 2

1081371

In [ ]:
train_dataset.write.parquet('./Data/train_trx_preprocessor.parquet', mode='overwrite')
test_dataset.write.parquet('./Data/test_trx_preprocessor.parquet', mode='overwrite')


In [11]:
df.write.parquet('./Data/trx_preprocessor.parquet', mode='overwrite')

24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_18 in memory! (computed 104.8 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_13 in memory! (computed 68.2 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_10 in memory! (computed 68.2 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_19 in memory! (computed 72.8 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_16 in memory! (computed 68.3 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_11 in memory! (computed 68.4 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_9 in memory! (computed 82.8 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_5 in memory! (computed 68.5 MiB so far)
24/06/13 21:04:08 WARN MemoryStore: Not enough space to cache rdd_155_8 in memory! (computed 68.2 MiB so far)
24/